In [ ]:
!pip install pathway --upgrade --quiet  # This cell may take a few seconds to execute.

In [ ]:
#importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import math


In [ ]:
!git clone https://github.com/Abhishek690012/Capstone_mini_project.git
df = pd.read_csv("Capstone_mini_project/dataset.csv")
df.head()

In [ ]:
#converting the year from  two digit form to four digit form
df['Year'] = df['LastUpdatedDate'].str[6:8]
df['LastUpdatedDate'] = df['LastUpdatedDate'].str[:-2]
df['Year'] = '20'+df['Year']
df['LastUpdatedDate' ]= df['LastUpdatedDate']+df['Year']
df['LastUpdatedTime'] = df['LastUpdatedTime'] + ':00'
df.drop(columns = ['Year'],inplace = True)
df.head()

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%m-%d-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)



> Model 1....



In [ ]:
# Savint the selected columns to a CSV file for streaming or downstream processing
df['code'] = df['SystemCodeNumber']
df[["Timestamp", "Occupancy", "Capacity","SystemCodeNumber","code"]].to_csv("parking_stream.csv", index=False)

class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    SystemCodeNumber : str
    code : str

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

#adding columns required to parse the input in a day wise manner
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
#Price function
def logistic_price(occ_max, occ_min, cap, base=10, scale=3):
    x = (occ_max - occ_min) / cap
    # Logistic function centered at 0.5, steepness )
    return base + scale / (1 + math.exp(-3 * (x - 0.5)))


In [ ]:
delta_window = (
    #the window creation based on the parking lot identity over one day
    data_with_time.windowby(
        pw.this.t,
        instance = pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(days = 1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy), #Max Occupancy during the day
        occ_min=pw.reducers.min(pw.this.Occupancy), #Min Occupancy during the day
        cap=pw.reducers.max(pw.this.Capacity),  #Capacity
        Code = pw.reducers.earliest(pw.this.SystemCodeNumber) #Adding the codes of parking lots to plow only one of them at a time
    )
    .with_columns(
        SystemCodeNumber = pw.this.Code,
        #applying the function to compute the price using the function defined above
        price=pw.apply(
        lambda occ_max, occ_min, cap: logistic_price(occ_max, occ_min, cap),
        pw.this.occ_max,
        pw.this.occ_min,
        pw.this.cap
      )
  )
)

In [ ]:
#upper limit or the index for plotting is 13 i.e. you can plot for 14 different parking lots
unique_codes = df['SystemCodeNumber'].unique()


# Activate the Panel extension to enable interactive visualizations
pn.extension()


# Creating the window in which the plots will be stored.
def price_plotter_Labeled(name):
    def price_plotter(source):
        from bokeh.plotting import figure
        #fig = figure(title=f"Price Plot for {name}")
        fig = bokeh.plotting.figure(
          height=300,
          width=600,
          title=f"Price Plot for {name}",
          x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
        )

        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.scatter(x="t", y="price", source=source, size=6, color="red", marker="circle")
        return fig
    return price_plotter

#upper limit or the index for plotting is 13 i.e. you can plot for 14 different parking lots
unique_codes = df['SystemCodeNumber'].unique()

plots = []#for storing the plots

for i in range(unique_codes.size):
  filtered = delta_window.filter(delta_window.SystemCodeNumber == unique_codes[i])
  viz = filtered.plot(price_plotter_Labeled(unique_codes[i]), sorting_col="t")
  plots.append(viz)


dashboard = pn.Column(*plots)
dashboard.servable()

In [ ]:
%%capture --no-display
pw.run()#Run this after creating the plots i.e. after doing run all run this code again
